In [1]:
import pandas as pd
import numpy as np
labels = pd.read_csv("/Users/mingliangang/Desktop/cifar10/trainLabels.csv")
labels.head()

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile


In [2]:
label2id = {x:i for i,x in enumerate(list(set(labels["label"])))}
id2label = {i:x for i,x in enumerate(list(set(labels["label"])))}
labelcon = lambda x: label2id[x]
labels['label'] = labels['label'].map(labelcon)
labels_ = labels['label'].as_matrix()

In [3]:
labels['label'].head()

0    3
1    5
2    5
3    0
4    8
Name: label, dtype: int64

In [4]:
labels_

array([3, 5, 5, ..., 5, 8, 8])

# Sanity check

In [5]:
labels['label'].map(lambda x: id2label[x]).head()

0          frog
1         truck
2         truck
3          deer
4    automobile
Name: label, dtype: object

# Tensorflow input pipeline

In [6]:
import os
from sklearn.model_selection import train_test_split
train = ["/Users/mingliangang/Desktop/cifar10/train/"+i for i in os.listdir("/Users/mingliangang/Desktop/cifar10/train")]
X_train, X_val, y_train, y_val = train_test_split(train, labels_, test_size=0.33, random_state=42)

In [7]:
import tensorflow as tf

NUM_CLASSES = 10 
def input_parser(img_path, label):
    # convert the label to one-hot encoding
    one_hot = tf.one_hot(label, NUM_CLASSES)

    # read the img from file
    img_file = tf.read_file(img_path)
    img_decoded = tf.image.decode_image(img_file, channels=3)

    return tf.cast(img_decoded,tf.float32), one_hot

In [8]:
from tensorflow.contrib.data import Dataset
tr_data = Dataset.from_tensor_slices((X_train, y_train))

In [9]:
tr_data = tr_data.map(input_parser)

In [26]:
batch_size = 128
batched_dataset = tr_data.batch(batch_size)
iterator = batched_dataset.make_initializable_iterator()
next_element = iterator.get_next()

In [27]:
sess = tf.Session()

# Resnet Model

In [29]:
from models import resnet
X = tf.placeholder("float", [batch_size, 32, 32, 3])
Y = tf.placeholder("float", [batch_size, 10])
learning_rate = tf.placeholder("float", [])
net = resnet(X,20)

Tensor("Shape_2:0", shape=(4,), dtype=int32)


In [38]:
cross_entropy = -tf.reduce_sum(Y*tf.log(net))
opt = tf.train.MomentumOptimizer(learning_rate, 0.9,use_nesterov=True)
train_op = opt.minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
#Multiple epochs
n_epochs = 1
for _ in range(n_epochs ):
  sess.run(iterator.initializer)
  while True:
    try:
      #There must be a better way tensor -> numpy -> tensor it seems kinda of stupid, there must be a better way to pipe. 
      loss,_ = sess.run([cross_entropy,train_op], feed_dict={X: sess.run(next_element[0]), Y: sess.run(next_element[1]),learning_rate: 0.001})
      print(loss)
    except ValueError:
      break

298.136
294.806
297.751
293.189
294.576
293.41
292.511
293.745
296.361
294.799
295.913
295.473
294.906
294.93
295.88
295.505
293.637
295.352
296.006
294.988
295.651
294.941
294.136
296.831
294.307
294.153
297.103
294.766
296.356
294.264
294.773
294.173
293.694
294.727
294.874
296.087
294.035
296.3
295.844
294.72
295.937
295.011
294.818
295.408
295.59
295.549
295.376
295.746
294.092
295.468
294.35
294.458
294.858
293.89
293.448
296.286
294.972
293.296
295.601
295.822
297.2
294.556
295.183
293.896
294.571
295.051
294.953
295.876
295.006
296.561
297.249
294.652
295.832
295.403
296.879
294.698
295.161
294.553
294.727
294.716
295.059
294.018
294.788
295.392
294.8
295.412
294.59
295.219
294.279
294.336
294.009
294.567
295.215
293.958
296.409
294.132
294.611
295.613
295.672
294.684
